In [ ]:
%load_ext autoreload
%autoreload 2
%store -r page

## 1.4 Editions per page with Wikiwho

In [ ]:
from wikiwho_wrapper import WikiWhoAPI, DataView as WikiWhoDV
import pandas as pd
api = WikiWhoAPI(lng='en')
wikiwho_dv = WikiWhoDV(api)
editions = wikiwho_dv.editor_content(page.page_id)
editions.head()

In [ ]:
df = editions
df['year_month'] = pd.to_datetime(df['year_month'])

# Group the data by year month and page (drop the editor information)
df.drop('editor_id', axis=1).groupby(['year_month','page_id']).sum()

# add columns with the total actions
df = df.join(pd.DataFrame(
    df.loc[:,'adds':'adds_stopword_count'].values +\
    df.loc[:,'dels':'dels_stopword_count'].values +\
    df.loc[:,'reins':'reins_stopword_count'].values, 
    index=df.index, 
    columns=['actions', 
             'actions_surv_48h', 
             'actions_persistent', 
             'actions_stopword_count']
))

# Visualization
from visualization.editions_listener import DFListener
from ipywidgets import interact
listener = DFListener(df)
# interact(listener.editions_per_month, 
#          begin=df.year_month,
#          end=df.year_month.sort_values(ascending=False),
#          actions=['All Actions', 'Additions', 'Reinsertions', 'Deletions'])
actions = df.loc[:,'actions':'actions_stopword_count'].columns.append(
    df.loc[:,'adds':'reins_stopword_count'].columns)
interact(listener.editions_per_month, 
         begin=df.year_month,
         end=df.year_month.sort_values(ascending=False),
         granularity=['Yearly', 'Monthly'],
         first_action=actions,
         second_action=actions)


## Conflict Score of Editors of a Page

In [ ]:
editors = editions['editor_id'].unique()
editors

In [ ]:
from wikiwho_wrapper import WikiWhoAPI, DataView as WikiWhoDV
from metrics.Conflict_Score import conflictScore_token_list

api = WikiWhoAPI(lng='en')
wikiwho_dv = WikiWhoDV(api)
revisions = wikiwho_dv.rev_ids_of_article(page.page_id)
tokens = wikiwho_dv.all_content(page.page_id)







In [ ]:
from metrics.Conflict_Score import conflictScore_token_list
conflictScore_token_list(tokens, 162969, revisions)

In [ ]:
revisions.head()

In [ ]:
tokens.loc[tokens['in'] == -1,'in'] = tokens.loc[tokens['in'] == -1,'o_rev_id']
tokens.head()

In [ ]:
#tokens[tokens['o_editor']=='25450560']

rev_ins = revisions[['rev_time', 'rev_id', 'o_editor']].rename(
    columns={'rev_time': 'in_rev_time',
             'rev_id': 'in',
             'o_editor': 'in_editor'}
)

rev_outs = revisions[['rev_time', 'rev_id', 'o_editor']].rename(
    columns={
        'rev_time': 'out_rev_time', 
        'rev_id': 'out',
        'o_editor': 'out_editor'
    }
)

fulltokens = pd.merge(tokens, rev_ins, how='left', on='in')
fulltokens = pd.merge(fulltokens, rev_outs, how='left', on='out')

fulltokens

In [ ]:
tokens.set_index('token_id')

possible_conflicts = tokens.groupby('token_id').size() > 3
possible_conflicts[possible_conflicts].index